In [1]:
import pandas as pd
# Read the CSV file into a pandas DataFrame
df = pd.read_csv("./handling_data.csv")

In [2]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

app = Flask(__name__)

X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')  # You can adjust the imputation strategy as needed
knn_model = KNeighborsClassifier(n_neighbors=1)  # You can adjust the number of neighbors (k) as needed

# Impute missing values in the training data
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
knn_model.fit(X_train_imputed, y_train)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if request.headers['Content-Type'] == 'application/json':
            data = request.get_json(force=True)

            if 'features' in data and isinstance(data['features'], dict):
                features_dict = data['features']

                # Order the features based on the training data columns
                features_ordered = [features_dict.get(feature, np.nan) for feature in X_train.columns]
                features_df = pd.DataFrame([features_ordered])

                # Impute missing values in the input data
                features_df_imputed = pd.DataFrame(imputer.transform(features_df), columns=features_df.columns)

                # Use the kNN model to make predictions
                prediction = knn_model.predict(features_df_imputed)

                return jsonify({'prediction': prediction.tolist()})
            else:
                return jsonify({'error': 'Invalid or missing "features" data in the request'}), 400
        else:
            return jsonify({'error': 'Invalid content type. Use application/json'}), 415
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return jsonify({'error': f'An internal server error occurred: {str(e)}'}), 500

if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Dec/2023 02:25:27] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2023 02:25:28] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2023 02:25:30] "GET / HTTP/1.1" 404 -
c:\Users\ktech\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\ktech\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [09/Dec/2023 02:26:22] "POST /predict HTTP/1.1" 200 -
